In [5]:
# dependencies
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import ComplementNB, MultinomialNB
import numpy as np
from scipy.stats import uniform, norm
from sklearn.metrics import plot_confusion_matrix
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn import svm
from XLB import *

import warnings
warnings.filterwarnings('ignore')

In [6]:
# extract data from files
x_train, y_train = extract_data("FinalTrainingSet.csv")
x_val, y_val = extract_data("Validation Set.csv")

# scale data values
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

# feature selection
num_features = 45
feat_sel = VarianceThreshold()
x_train = feat_sel.fit_transform(x_train)
# feat_sel_2 = SelectFromModel(estimator=DecisionTreeClassifier(random_state=481516234))
# feat_sel_2 = SelectFromModel(\
#                 estimator=RandomForestClassifier(n_estimators=50,\
#                                              random_state=481516234))
# 33.33% 0.24
feat_sel_2 = SelectFromModel(\
                estimator=LogisticRegression(random_state=481516234))
# feat_sel_2 = SelectFromModel(\
#                 estimator=svm.LinearSVC(C=0.25, penalty="l1", dual=False,\
#                                     random_state=481516234))
# feat_sel_2 = SelectKBest(mutual_info_classif,k=num_features)
x_train = feat_sel_2.fit_transform(x_train,y_train)
x_val = feat_sel_2.transform(feat_sel.transform(x_val))
# print("After Variance Threshold Feature Selection:",x_train.shape)

rand_seed = 3454132

oversampler = SMOTE(sampling_strategy="not majority",random_state=rand_seed)
x_smote, y_smote = oversampler.fit_resample(x_train,y_train)
print(x_smote.shape,y_smote.shape)

oversampler = RandomOverSampler(sampling_strategy="not majority",random_state=rand_seed)
x_os, y_os = oversampler.fit_resample(x_train,y_train)
print(x_os.shape,y_os.shape)

(750, 69) (750,)
(750, 69) (750,)


In [8]:
#MOVING FEATURE HEADERS INTO A LIST
import csv

with open("FinalTrainingSet.csv") as f:
    reader = csv.reader(f)
    features = next(reader)

#Deleting everything except features from the dataset
features.remove("Row Labels")
features.remove("Theme")
features.remove("Theme(Numbered)")
print(len(features))
#Retained features after selection
selected_feats = feat_sel_2.get_support(True)

for ind, ft in sorted(enumerate(features), reverse=True): 
    if ind not in selected_feats:
        del features[ind]
print(len(features))
#Printing modified list 
#print (features) 
#print (len(features))

324
69


In [40]:
calm = []
cheerful = []
bravery = [] 
fearful = [] 
love = [] 
sadness = []
themes = ["Calm","Cheerful","Bravery","Fearful","Love","Sadness"]
for f in range(0,len(features)):    
    calm1 = []
    cheerful1 = []
    bravery1 = [] 
    fearful1 = [] 
    love1 = [] 
    sadness1 = []
    feats = []
    for d in range(0, len(x_train)):
        if y_train[d] == 1:
            calm1.append(x_train[f])
        if y_train[d] == 2:
            cheerful1.append(x_train[f])
        if y_train[d] == 3:
            bravery1.append(x_train[f])
        if y_train[d] == 4:
            fearful1.append(x_train[f])
        if y_train[d] == 5:
            love1.append(x_train[f])
        if y_train[d] == 6:
            sadness1.append(x_train[f])
    calm.append(calm1)
    cheerful.append(cheerful1)
    bravery.append(bravery1)
    fearful.append(fearful1)
    love.append(love1)
    sadness.append(sadness1)
print(x_train)

[[0.         0.         1.         ... 0.2919568  0.17662811 0.17661486]
 [1.         1.         1.         ... 0.23618019 0.17934551 0.1793323 ]
 [1.         0.         1.         ... 0.25145659 0.18423683 0.1842237 ]
 ...
 [1.         0.         1.         ... 0.3711809  0.20217166 0.20215882]
 [1.         0.         0.         ... 0.34489129 0.20543254 0.20541975]
 [1.         0.         0.         ... 0.30154896 0.20814994 0.2081372 ]]


In [39]:
variance = []
for i in range(0, len(calm)):
    var = []
    var.append(np.var(calm[i]))
    var.append(np.var(cheerful[i]))
    var.append(np.var(bravery[i]))
    var.append(np.var(fearful[i]))
    var.append(np.var(love[i]))
    var.append(np.var(sadness[i]))
    variance.append(var)
print(variance)

[[0.07780421534710674, 0.07780421534710674, 0.07780421534710673, 0.07780421534710673, 0.07780421534710674, 0.07780421534710674], [0.0863414807399268, 0.0863414807399268, 0.08634148073992678, 0.08634148073992681, 0.0863414807399268, 0.0863414807399268], [0.08019532199811899, 0.080195321998119, 0.08019532199811902, 0.08019532199811899, 0.080195321998119, 0.080195321998119], [0.07380141644757252, 0.0738014164475725, 0.0738014164475725, 0.0738014164475725, 0.07380141644757252, 0.0738014164475725], [0.07730338244808066, 0.07730338244808067, 0.07730338244808066, 0.07730338244808066, 0.07730338244808067, 0.07730338244808065], [0.07356181431859116, 0.07356181431859116, 0.07356181431859114, 0.07356181431859114, 0.07356181431859114, 0.07356181431859116], [0.06895674551448384, 0.06895674551448384, 0.06895674551448384, 0.06895674551448383, 0.06895674551448383, 0.06895674551448383], [0.050420011633258294, 0.050420011633258294, 0.05042001163325829, 0.0504200116332583, 0.050420011633258294, 0.0504200